# Visualize Emotion Trajectories

In [182]:
%%HTML
<style>
    body{
 --vscode-font-family: "ComicShannsMono Nerd Font";
    }
</style>

## Import Libraries

In [183]:
import ast
import json
import random
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from scipy.stats import entropy, skew, pearsonr
from scipy.interpolate import interp1d
from plotly.subplots import make_subplots


random.seed(2) # entropy high
# random.seed(3) # entropy low
# random.seed(4) # entropy low
# random.seed(5)  # entropy medium
# random.seed(6) # story to short
# random.seed(7) # missing value
# random.seed(8) 
# random.seed(9)
# random.seed(12)

## Load Data

In [184]:
plutchik_emotions = [
    "anger",
    "anticipation",
    "disgust",
    "fear",
    "joy",
    "sadness",
    "surprise",
    "trust",
]

In [185]:
human_story_path = "../data/human_story_emotion_scored.csv"
model_story_path = "../data/model_story_emotion_scored.csv"

human_story_df = pd.read_csv(human_story_path)
model_story_df = pd.read_csv(model_story_path)

Convert the string back into list

In [186]:
for emotion in plutchik_emotions:
    human_story_df[emotion] = human_story_df[emotion].apply(ast.literal_eval)
    model_story_df[emotion] = model_story_df[emotion].apply(ast.literal_eval)

In [187]:
human_story_df.head()

,Unnamed: 0,Prompt,Story,Model,Length,Sentences,Sentences Length,anger,anticipation,joy,trust,fear,surprise,sadness,disgust
0,0,When you die the afterlife is an arena where y...,"3,000 years have I been fighting. Every mornin...",Human,1076,"['3,000 years have I been fighting.', 'Every m...",21,"[0.458, 0.438, 0.438, 0.29, 0.375, 0.23, 0.5, ...","[0.292, 0.354, 0.375, 0.29, 0.354, 0.32, 0.29,...","[0.2, 0.292, 0.2, 0.23, 0.29, 0.2, 0.2, 0.25, ...","[0.29, 0.29, 0.27, 0.27, 0.375, 0.34, 0.2, 0.2...","[0.4, 0.438, 0.438, 0.354, 0.438, 0.396, 0.5, ...","[0.458, 0.438, 0.438, 0.438, 0.396, 0.438, 0.3...","[0.457, 0.438, 0.438, 0.354, 0.438, 0.396, 0.4...","[0.4, 0.438, 0.438, 0.292, 0.354, 0.292, 0.438..."
1,1,A new law is enacted that erases soldiers memo...,"“Dad, you 're on TV again !” I heard Eric 's v...",Human,1315,"[""“Dad, you 're on TV again !” I heard Eric 's...",17,"[0.375, 0.375, 0.396, 0.396, 0.375, 0.375, 0.2...","[0.396, 0.43, 0.479, 0.396, 0.352, 0.354, 0.35...","[0.48, 0.23, 0.396, 0.292, 0.23, 0.23, 0.34, 0...","[0.46, 0.354, 0.438, 0.396, 0.375, 0.392, 0.46...","[0.34, 0.479, 0.479, 0.396, 0.46, 0.438, 0.396...","[0.479, 0.375, 0.58, 0.396, 0.458, 0.438, 0.37...","[0.354, 0.438, 0.396, 0.354, 0.438, 0.396, 0.3...","[0.396, 0.4, 0.438, 0.333, 0.396, 0.396, 0.292..."
2,2,A scientific study proves that all humans have...,"When Tyler entered the ward, his daughter Vale...",Human,4420,"['When Tyler entered the ward, his daughter Va...",44,"[0.375, 0.292, 0.375, 0.375, 0.396, 0.438, 0.3...","[0.375, 0.34, 0.396, 0.438, 0.326, 0.375, 0.43...","[0.27, 0.375, 0.462, 0.292, 0.25, 0.2, 0.2, 0....","[0.396, 0.438, 0.438, 0.396, 0.354, 0.396, 0.3...","[0.479, 0.292, 0.396, 0.396, 0.395, 0.438, 0.4...","[0.438, 0.375, 0.458, 0.438, 0.396, 0.462, 0.3...","[0.479, 0.396, 0.396, 0.523, 0.438, 0.438, 0.4...","[0.396, 0.294, 0.396, 0.396, 0.396, 0.396, 0.3..."
3,3,Write a story about an elderly wizard and his ...,His body was failing. He had taken care of it ...,Human,4575,"['His body was failing.', 'He had taken care o...",58,"[0.292, 0.375, 0.292, 0.292, 0.396, 0.292, 0.3...","[0.29, 0.396, 0.375, 0.354, 0.375, 0.396, 0.35...","[0.2, 0.34, 0.23, 0.29, 0.23, 0.23, 0.274, 0.2...","[0.29, 0.438, 0.396, 0.438, 0.396, 0.354, 0.37...","[0.396, 0.3, 0.292, 0.27, 0.46, 0.375, 0.375, ...","[0.396, 0.438, 0.396, 0.354, 0.375, 0.438, 0.3...","[0.438, 0.396, 0.438, 0.375, 0.438, 0.438, 0.4...","[0.354, 0.3, 0.354, 0.292, 0.396, 0.354, 0.354..."
4,4,"You have become death, destroyer of worlds.","I saw the button. It was simple, red, no words...",Human,842,"['I saw the button.', 'It was simple, red, no ...",11,"[0.292, 0.375, 0.375, 0.458, 0.375, 0.292, 0.2...","[0.292, 0.396, 0.396, 0.4, 0.39, 0.438, 0.396,...","[0.292, 0.354, 0.396, 0.2, 0.27, 0.34, 0.438, ...","[0.27, 0.46, 0.4, 0.29, 0.396, 0.438, 0.438, 0...","[0.292, 0.396, 0.396, 0.58, 0.479, 0.396, 0.37...","[0.396, 0.396, 0.438, 0.479, 0.46, 0.27, 0.375...","[0.292, 0.396, 0.396, 0.58, 0.438, 0.354, 0.39...","[0.292, 0.396, 0.354, 0.458, 0.396, 0.292, 0.2..."


### Data Preprocessing

Remove missing values

In [188]:
def remove_missing_values(emotion_list):
    return [emotion for emotion in emotion_list if emotion != -1]
    
for emotion in plutchik_emotions:
    human_story_df[emotion] = human_story_df[emotion].apply(remove_missing_values)
    model_story_df[emotion] = model_story_df[emotion].apply(remove_missing_values)

Normalization across time steps

In [189]:
NORMALIZE_STEP = 50
def normalize_emotion_sequences(emotion_sequences, n_steps):
    """ Normalize emotion score sequences to have the same length """
    if len(emotion_sequences) == 0:
        assert Exception("Emotion sequences should not be empty")
    f = interp1d(np.linspace(0, 1, len(emotion_sequences)), emotion_sequences, kind="linear", fill_value="extrapolate")
    return f(np.linspace(0, 1, n_steps)).tolist()

# normalize each emotion sequence to have the same length
for emotion in plutchik_emotions:
    human_story_df[emotion] = human_story_df[emotion].apply(normalize_emotion_sequences, n_steps=NORMALIZE_STEP)
    model_story_df[emotion] = model_story_df[emotion].apply(normalize_emotion_sequences, n_steps=NORMALIZE_STEP)

/home/master/13/r13922002/miniconda3/envs/ASE/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:479: RuntimeWarning:

invalid value encountered in divide



### Emotion strength normalization

In [190]:
# def normalized_emotion_strength_in_time_step(emotion_sequences):
#     total_strength = sum(emotion_sequences)
#     probabilities = [strength / total_strength for strength in emotion_sequences]
#     return softmax(probabilities)

# for i, row in human_story_df.iterrows():
#     for time_step_idx in range(NORMALIZE_STEP):
#         normalized_emotion_intensity = normalized_emotion_strength_in_time_step([row[emotion][time_step_idx] for emotion in plutchik_emotions])
#         for emotion_idx, emotion in enumerate(plutchik_emotions):
#             human_story_df.at[i, emotion][time_step_idx] = normalized_emotion_intensity[emotion_idx]

# for i, row in model_story_df.iterrows():
#     for time_step_idx in range(NORMALIZE_STEP):
#         normalized_emotion_intensity = normalized_emotion_strength_in_time_step([row[emotion][time_step_idx] for emotion in plutchik_emotions])
#         for emotion_idx, emotion in enumerate(plutchik_emotions):
#             model_story_df.at[i, emotion][time_step_idx] = normalized_emotion_intensity[emotion_idx]

In [191]:
human_story_df.head()

,Unnamed: 0,Prompt,Story,Model,Length,Sentences,Sentences Length,anger,anticipation,joy,trust,fear,surprise,sadness,disgust
0,0,When you die the afterlife is an arena where y...,"3,000 years have I been fighting. Every mornin...",Human,1076,"['3,000 years have I been fighting.', 'Every m...",21,"[0.458, 0.4498367346938776, 0.4416734693877551...","[0.292, 0.3173061224489796, 0.3426122448979591...","[0.2, 0.23379591836734692, 0.2675918367346939,...","[0.29, 0.29, 0.29, 0.28551020408163263, 0.2773...","[0.4, 0.41551020408163264, 0.4310204081632653,...","[0.458, 0.4498367346938776, 0.4416734693877551...","[0.457, 0.4492448979591837, 0.4414897959183673...","[0.4, 0.41551020408163264, 0.4310204081632653,..."
1,1,A new law is enacted that erases soldiers memo...,"“Dad, you 're on TV again !” I heard Eric 's v...",Human,1315,"[""“Dad, you 're on TV again !” I heard Eric 's...",17,"[0.375, 0.375, 0.375, 0.375, 0.381428571428571...","[0.396, 0.40710204081632656, 0.418204081632653...","[0.48, 0.3983673469387755, 0.31673469387755104...","[0.46, 0.42538775510204085, 0.3907755102040816...","[0.34, 0.3853877551020408, 0.4307755102040816,...","[0.479, 0.4450408163265306, 0.4110816326530612...","[0.354, 0.3814285714285714, 0.4088571428571428...","[0.396, 0.3973061224489796, 0.3986122448979592..."
2,2,A scientific study proves that all humans have...,"When Tyler entered the ward, his daughter Vale...",Human,4420,"['When Tyler entered the ward, his daughter Va...",44,"[0.375, 0.3021632653061224, 0.3546734693877550...","[0.375, 0.3442857142857143, 0.3822857142857143...","[0.27, 0.36, 0.43714285714285717, 0.3648571428...","[0.396, 0.43285714285714283, 0.438, 0.41142857...","[0.479, 0.31489795918367347, 0.370530612244897...","[0.438, 0.38271428571428573, 0.437673469387755...","[0.479, 0.40616326530612246, 0.396, 0.47634693...","[0.396, 0.30648979591836734, 0.371020408163265..."
3,3,Write a story about an elderly wizard and his ...,His body was failing. He had taken care of it ...,Human,4575,"['His body was failing.', 'He had taken care o...",58,"[0.292, 0.36144897959183675, 0.292, 0.34293877...","[0.29, 0.39257142857142857, 0.3681428571428571...","[0.2, 0.32204081632653064, 0.24959183673469387...","[0.29, 0.43114285714285716, 0.4097142857142857...","[0.396, 0.2986938775510204, 0.2848163265306122...","[0.396, 0.43114285714285716, 0.382285714285714...","[0.438, 0.40285714285714286, 0.417428571428571...","[0.354, 0.30881632653061225, 0.333755102040816..."
4,4,"You have become death, destroyer of worlds.","I saw the button. It was simple, red, no words...",Human,842,"['I saw the button.', 'It was simple, red, no ...",11,"[0.292, 0.30893877551020404, 0.325877551020408...","[0.292, 0.3132244897959183, 0.3344489795918367...","[0.292, 0.3033877551020408, 0.3147755102040816...","[0.27, 0.30877551020408167, 0.3475510204081632...","[0.292, 0.3132244897959183, 0.3344489795918367...","[0.396, 0.396, 0.396, 0.396, 0.396, 0.39685714...","[0.292, 0.3132244897959183, 0.3344489795918367...","[0.292, 0.3132244897959183, 0.3344489795918367..."


## Visualization

* Randomly select a prompt to compare human-written stories' emotion trajectories with the AI-generated stories' emotion trajectories.

In [192]:
def get_random_story_example():
    random_prompt = random.choice(human_story_df["Prompt"])
    human_story_example = human_story_df[human_story_df["Prompt"] == random_prompt]
    model_story_examples = model_story_df[model_story_df["Prompt"] == random_prompt]
    return human_story_example, model_story_examples, random_prompt

human_story_example, model_story_examples, prompt = get_random_story_example()
concated_example_df = pd.concat([human_story_example, model_story_examples])

### Emotion Trajectories Visualization (Single Story)

In [193]:
def draw_human_model_emotion_trajectories(human_story_example, model_story_examples, prompt):
    # Create a 1x2 subplot
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Human-written story", "Model-generated story"))

    # Add Human traces (group by emotion)
    human_story_example = human_story_example.iloc[0]
    for emotion in plutchik_emotions:
        score_list = human_story_example[emotion]
        score_list_len = len(score_list)
        fig.add_trace(
            go.Scatter(
                x=list(range(1, score_list_len+1)),
                y=score_list,
                mode="lines+markers",
                name=f"Human - {emotion}",
                legendgroup="Human",
                visible="legendonly"
            ),
            row=1,
            col=1,
        )

    # Add Model traces (group by emotion)
    for _, model_story_example in model_story_examples.iterrows():
        model_name = model_story_example["Model"]
        for emotion in plutchik_emotions:
            score_list = model_story_example[emotion]
            score_list_len = len(score_list)
            fig.add_trace(
                go.Scatter(
                    x=list(range(1, score_list_len+1)),
                    y=score_list,
                    mode="lines+markers",
                    name=f"{model_name} - {emotion}",
                    legendgroup=model_name,
                    visible="legendonly"
                ),
                row=1,
                col=2,
            )

    # Update layout with interactive legend settings
    fig.update_layout(
        height=600,
        title=f"Human vs Model emotion trajectories - {prompt}",
        template="plotly_dark",
        yaxis=dict(range=[0, 1]),
        yaxis2=dict(range=[0, 1]),
        legend=dict(
            groupclick="toggleitem",
        ),
    )

    fig.show()


draw_human_model_emotion_trajectories(human_story_example, model_story_examples, prompt)

## Single Story Analysis

In [194]:
# View the data
concated_example_df.head()

,Unnamed: 0,Prompt,Story,Model,Length,Sentences,Sentences Length,anger,anticipation,joy,trust,fear,surprise,sadness,disgust
7,7,Two people promise their first born child to t...,"**Prelude: ** “Brujeria ,” they said, looking ...",Human,4383,"['**Prelude: ** “Brujeria ,” they said, lookin...",53,"[0.375, 0.375, 0.36483673469387756, 0.30724489...","[0.354, 0.396, 0.39648979591836736, 0.39926530...","[0.292, 0.37885714285714284, 0.408857142857142...","[0.396, 0.3985714285714286, 0.4328571428571429...","[0.375, 0.396, 0.40616326530612246, 0.47146938...","[0.396, 0.43689795918367347, 0.414489795918367...","[0.375, 0.396, 0.40114285714285713, 0.438, 0.4...","[0.396, 0.396, 0.383265306122449, 0.3111020408..."
7,7,Two people promise their first born child to t...,Great! Please provide me with more details li...,Llama-7b,4156,"[' Great!', 'Please provide me with more detai...",44,"[0.292, 0.38326530612244897, 0.396, 0.33020408...","[0.32, 0.45953061224489794, 0.4163265306122449...","[0.438, 0.402, 0.426, 0.41400000000000003, 0.3...","[0.438, 0.438, 0.48481632653061224, 0.43420408...","[0.292, 0.42012244897959183, 0.407040816326530...","[0.396, 0.46883673469387754, 0.416326530612244...","[0.292, 0.38326530612244897, 0.396, 0.38271428...","[0.292, 0.38326530612244897, 0.364285714285714..."
103,103,Two people promise their first born child to t...,"\n\nOnce upon a time, in a small village nestl...",Mistral-7b,3306,"['\n\nOnce upon a time, in a small village nes...",28,"[0.29, 0.33683673469387754, 0.3602040816326530...","[0.292, 0.3493061224489796, 0.3902857142857143...","[0.29, 0.32526530612244897, 0.3578775510204081...","[0.396, 0.41914285714285715, 0.438, 0.438, 0.4...","[0.396, 0.396, 0.38518367346938776, 0.32677551...","[0.396, 0.396, 0.396, 0.396, 0.404571428571428...","[0.29, 0.37155102040816324, 0.4228979591836734...","[0.29, 0.3484081632653061, 0.376, 0.268, 0.231..."
199,199,Two people promise their first born child to t...,"""The Bargain of Two Souls""\n\nOnce upon a tim...",Beluga-13b,2775,"[' ""The Bargain of Two Souls""\n\nOnce upon a t...",24,"[0.29, 0.2909387755102041, 0.29187755102040813...","[0.36, 0.3787755102040816, 0.39755102040816326...","[0.29, 0.28061224489795916, 0.2712244897959184...","[0.375, 0.38673469387755105, 0.398469387755102...","[0.438, 0.4084285714285714, 0.3788571428571428...","[0.396, 0.4157142857142857, 0.4354285714285714...","[0.396, 0.396, 0.396, 0.37885714285714284, 0.3...","[0.375, 0.3360408163265306, 0.2970816326530612..."
295,295,Two people promise their first born child to t...,"Once upon a time, in a quiet little village, ...",OrcaPlatypus-13b,2934,"[' Once upon a time, in a quiet little village...",31,"[0.29, 0.2777551020408163, 0.2935714285714286,...","[0.292, 0.30914285714285716, 0.337061224489795...","[0.34, 0.4524489795918367, 0.4748979591836735,...","[0.396, 0.5698775510204082, 0.611530612244898,...","[0.29, 0.25326530612244896, 0.2625510204081632...","[0.29, 0.3083673469387755, 0.34648979591836737...","[0.29, 0.29122448979591836, 0.3247755102040816...","[0.292, 0.2356734693877551, 0.2206530612244898..."


## 1. Mean-Std Analysis 

* Mean: emotion's average intensity across whole story
* Std: emotion's intensity variation or consistency across whole story

In [195]:
def visualize_mean_std(df, emotions):
    stats = {emotion: {"mean": [], "std": []} for emotion in emotions}
    for _, row in df.iterrows():
        for emotion in emotions:
            values = np.array(row[emotion])
            stats[emotion]["mean"].append(np.mean(values))
            stats[emotion]["std"].append(np.std(values))

    fig = make_subplots(rows=1, cols=1, subplot_titles=["Mean ± Std"])
    for emotion in emotions:
        fig.add_trace(go.Bar(
            x=["Human" if row["Model"] == "Human" else row["Model"] for _, row in df.iterrows()],
            y=stats[emotion]["mean"],
            error_y=dict(type='data', array=stats[emotion]["std"]),
            name=f"{emotion.capitalize()} (Mean ± Std)"
        ), row=1, col=1)
    fig.update_layout(
        height=600,
        title="Emotion Mean ± Std Across Stories",
        xaxis_title="Story Source (Human vs Model)",
        yaxis_title="Emotion Intensity",
        template="plotly_dark",
        yaxis=dict(range=[0.2, 0.6], tick0=0.2, dtick=0.05)  
    )
    fig.show()
    return stats

mean_std_stats = visualize_mean_std(concated_example_df, plutchik_emotions)

In [196]:
print(json.dumps(mean_std_stats, indent=4))

{
    "anger": {
        "mean": [
            0.33636897959183676,
            0.3455118367346939,
            0.30892040816326527,
            0.3238159183673469,
            0.3311804081632653,
            0.31507755102040813,
            0.3181767346938775
        ],
        "std": [
            0.04036943079576234,
            0.0533168571933361,
            0.04454512647168234,
            0.05235966897600641,
            0.045415880538432936,
            0.04187603350872656,
            0.048995095099136486
        ]
    },
    "anticipation": {
        "mean": [
            0.3872375510204082,
            0.3945236734693877,
            0.3747926530612245,
            0.3869126530612245,
            0.3666734693877551,
            0.3583595918367347,
            0.3609865306122449
        ],
        "std": [
            0.03260647793509665,
            0.040395429576816556,
            0.045772791344733894,
            0.03455850468983482,
            0.0490546161802986,
      

## 2. Emotion Diversity Analysis

Using **Entropy** to measure the diversity of emotions in the story
* High entropy: more uniform the distribution of emotions       -> higher diversity
* Low entropy : more concentrated the distribution of emotions  -> lower diversity

In [197]:
def calculate_entropy_for_time_step(emotion_strengths):
    """ Calculate the entropy of the emotion distribution at a time step """
    # Normalize the emotion strengths to get a probability distribution
    total_strength = sum(emotion_strengths)
    probabilities = [strength / total_strength for strength in emotion_strengths]
    # Calculate the entropy
    entropy = -sum([p * np.log2(p) for p in probabilities if p > 0])
    return entropy


def visualize_emotion_diversity():
    story_entropy_dict = {}
    for _, row in concated_example_df.iterrows():
        time_step_entropies = []
        for time_step_idx in range(NORMALIZE_STEP):
            # get all emotions' intensity at the current time step
            time_step_strengths = [row[emotion][time_step_idx] for emotion in plutchik_emotions]
            # calculate the entropy of the emotion distribution at the current time step
            time_step_entropy = calculate_entropy_for_time_step(time_step_strengths)
            time_step_entropies.append(time_step_entropy)
        # calculate the whole story's average emotion diversity entropy (Average entropy across all time steps)
        total_entropy = np.mean(time_step_entropies)
        story_entropy_dict[row["Model"]] = total_entropy
    
    # Create a bar plot for the emotion diversity entropy
    fig = go.Figure()
    for model_name, entropy_value in story_entropy_dict.items():
        fig.add_trace(go.Bar(
            x=[model_name],
            y=[entropy_value],
            name=f"{model_name} - Emotion Diversity Entropy"
        ))
    fig.update_layout(
        height=600,
        title="Emotion Diversity Entropy Across Stories",
        xaxis_title="Story Source (Human vs Model)",
        yaxis_title="Emotion Diversity Entropy (Log Scale)",
        yaxis=dict(type="log"),
        template="plotly_dark",
        showlegend=True
    )
    fig.show()
    return story_entropy_dict

entropy_stats = visualize_emotion_diversity()

## 3. Skewness Analysis

* High positive skewness (right-skewed): higher emotional intensity intensity values are more common than low emotional intensity values
* High negative skewness (left-skewed): lower emotional intensity intensity values are more common than high emotional intensity values
* Around 0 skewness: symmetric distribution

In [198]:
def visualize_skewness():
    stats = {emotion: {"skewness": []} for emotion in plutchik_emotions}
    for _, row in concated_example_df.iterrows():
        for emotion in plutchik_emotions:
            values = np.array(row[emotion])
            stats[emotion]["skewness"].append(skew(values))

    fig = make_subplots(rows=1, cols=1, subplot_titles=["Skewness"])
    for emotion in plutchik_emotions:
        fig.add_trace(go.Bar(
            x=["Human" if row["Model"] == "Human" else row["Model"] for _, row in concated_example_df.iterrows()],
            y=stats[emotion]["skewness"],
            # mode="lines+markers",
            name=f"{emotion.capitalize()} Skewness",
            text=[f'{s:.2f}' for s in stats[emotion]["skewness"]],
            textposition="auto"
        ), row=1, col=1)
    fig.update_layout(
        height=600,
        title="Emotion Skewness Across Stories",
        xaxis_title="Story Source (Human vs Model)",
        yaxis_title="Skewness",
        template="plotly_dark",
        barmode="group",
        showlegend=True
    )
    fig.show()
    return stats

skewness_stats = visualize_skewness()

## 4. Emotion Correlation Analysis

In [231]:
def visualize_correlation_matrix():
    fig = make_subplots(rows=1, cols=2, subplot_titles=["Human", "Model"])
    for i, row in concated_example_df.iterrows():
        tmp_dict = row[plutchik_emotions].to_dict()
        emotion_df = pd.DataFrame(tmp_dict)
        correlation_matrix = emotion_df.corr()
        fig.add_trace(go.Heatmap(
            z=correlation_matrix.values,
            x=plutchik_emotions,
            y=plutchik_emotions,
            name=row["Model"],
            colorscale="RdBu",
            coloraxis="coloraxis",
            showscale=(row["Model"] == "Human")
        ), row=1, col=1 if row["Model"] == "Human" else 2)
    buttons = [
        dict(
            label=model,
            method="update",
            args=[
                {
                    "visible": [
                        True if i == idx or concated_example_df['Model'].iloc[i] == "Human" else False
                        for i in range(len(concated_example_df))
                    ]
                }
            ]
        )
        for idx, model in enumerate(concated_example_df["Model"].unique()) if model != "Human"
    ]
    fig.update_layout(
        coloraxis=dict(
            colorscale="RdBu",
            cmin=-1,
            cmax=1,
            colorbar=dict(
                title="Correlation",
                thickness=20,
                len=0.8,
                x=1.02
            )
        ),
        updatemenus=[
            dict(
                buttons=buttons,
                direction="down",
                showactive=True,
                x=0.9,
                y=1.2
            )
        ],
        title="Emotion Correlation Matrix Across Models",
        height=600,
        width=1100,
        template="plotly_dark"
    )
    fig.show()

visualize_correlation_matrix()